[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/QuantLet/EMQA/blob/main/EN/quantlets/EMQA_seasonality/EMQA_seasonality.ipynb)

# EMQA_seasonality

Detecting seasonality: visual and ACF methods.

**Output:** `seasonality_detection.pdf`

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.rcParams.update({
    'figure.facecolor': 'none',
    'axes.facecolor': 'none',
    'savefig.facecolor': 'none',
    'savefig.transparent': True,
    'axes.grid': False,
    'axes.spines.top': False,
    'axes.spines.right': False,
    'font.size': 11,
    'figure.figsize': (12, 6),
})

COLORS = {
    'blue': '#1A3A6E', 'red': '#CD0000', 'green': '#2E7D32',
    'orange': '#E67E22', 'purple': '#8E44AD', 'gray': '#808080',
    'cyan': '#00BCD4', 'amber': '#B5853F'
}

def save_fig(fig, name):
    fig.savefig(name, bbox_inches='tight', transparent=True, dpi=300)
    print(f"Saved: {name}")


In [ ]:
import yfinance as yf

def fetch(ticker, start='2020-01-01', end='2025-12-31'):
    d = yf.download(ticker, start=start, end=end, progress=False)
    if isinstance(d.columns, pd.MultiIndex):
        return d['Close'].squeeze().dropna()
    return d['Close'].dropna()


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf

gas = fetch('NG=F')
monthly = gas.resample('ME').mean().dropna()

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

for year in monthly.index.year.unique():
    subset = monthly[monthly.index.year == year]
    ax1.plot(subset.index.month, subset.values, 'o-', lw=1, markersize=4, label=str(year))
ax1.set_xticks(range(1,13))
ax1.set_xticklabels(['J','F','M','A','M','J','J','A','S','O','N','D'])
ax1.set_title('Year-over-Year Overlay', fontweight='bold')
ax1.set_ylabel('USD/MMBtu')
ax1.legend(loc='upper center', bbox_to_anchor=(0.5, -0.12), ncol=6, frameon=False)

plot_acf(monthly, ax=ax2, lags=36, alpha=0.05)
ax2.set_title('ACF: Monthly Gas (spikes at lag 12, 24)', fontweight='bold')
ax2.axvline(12, color=COLORS['red'], ls='--', alpha=0.5)
ax2.axvline(24, color=COLORS['red'], ls='--', alpha=0.5)

fig.tight_layout()
save_fig(fig, 'seasonality_detection.pdf')
plt.show()
